In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
try:
    sc.stop()
except:
    print("SparkContext has not been initiated")

SparkContext has not been initiated


In [3]:
conf = SparkConf() \
    .set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.postgresql:postgresql:42.5.0") \

sc = SparkContext(conf=conf)

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("myApp") \
    .getOrCreate()

spark

In [4]:
club_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('uri', "mongodb://mongodb:27017/football_data.club") \
    .load()
    
club_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- ClubName: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- std: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Player: string (nullable = true)
 |    |    |    |-- Nation: string (nullable = true)
 |    |    |    |-- Pos: string (nullable = true)
 |    |    |    |-- Age: string (nullable = true)
 |    |    |    |-- 90s: string (nullable = true)
 |    |    |    |-- Gls: string (nullable = true)
 |    |    |    |-- Sh: void (nullable = true)
 |    |    |    |-- SoT: string (nullable = true)
 |    |    |    |-- SoT%: void (nullable = true)
 |    |    |    |-- Sh/90: void (nullable = true)
 |    |    |    |-- SoT/90: string (nullable = true)
 |    |    |    |-- G/Sh: void (nullable = true)
 |    |    |    |-- G/SoT: string (nullable = true)
 |    |    |    |-- Dist: void (nullable = true)
 |    |    |    |-- PK: string (nullable = tr

In [6]:
club_df.show()

+--------------------+--------------------+--------------------+----+
|                 _id|                info|               stats|type|
+--------------------+--------------------+--------------------+----+
|1998-1999##Salama...|{1998-1999Salaman...|{[{/en/players/d8...|club|
|1999-2000##Sevill...|{1999-2000Sevilla...|{[{/en/players/48...|club|
|2000-2001##Numanc...|{2000-2001Numanci...|{[{/en/players/41...|club|
|2001-2002##Zarago...|{2001-2002Zaragoz...|{[{/en/players/e5...|club|
|2002-2003##Rayo-V...|{2002-2003RayoVal...|{[{/en/players/b4...|club|
|2003-2004##Real-M...|{2003-2004RealMur...|{[{/en/players/70...|club|
|2004-2005##Albace...|{2004-2005Albacet...|{[{/en/players/9a...|club|
|2005-2006##Malaga...|{2005-2006MálagaS...|{[{/en/players/23...|club|
|2006-2007##Gimnas...|{2006-2007Gimnàst...|{[{/en/players/99...|club|
|2007-2008##Levant...|{2007-2008Levante...|{[{/en/players/f4...|club|
|2008-2009##Recrea...|{2008-2009Recreat...|{[{/en/players/58...|club|
|2009-2010##Xerez-..

In [5]:
club = club_df.toPandas()
club

,_id,info,stats,type
0,1998-1999##Salamanca-Stats,"(1998-1999SalamancaStats,)","([(/en/players/d859d8c2/Martin-Vellisca, /en/c...",club
1,1999-2000##Sevilla-Stats,"(1999-2000SevillaStats,)","([(/en/players/48ddb821/Carlos-Marchena, /en/c...",club
2,2000-2001##Numancia-Stats,"(2000-2001NumanciaStats,)","([(/en/players/41a60718/Txomin-Nagore, /en/cou...",club
3,2001-2002##Zaragoza-Stats,"(2001-2002ZaragozaStats,)","([(/en/players/e52392a7/Paco, /en/country/ESP/...",club
4,2002-2003##Rayo-Vallecano-Stats,"(2002-2003RayoVallecanoStats,)","([(/en/players/b4a2cf8b/Patricio-Graff, /en/co...",club
...,...,...,...,...
97,2009-2010##Zaragoza-Stats,"(2009-2010ZaragozaStats,)","([(/en/players/6f3a6343/Leonardo-Ponzio, /en/c...",club
98,2009-2010##Almeria-Stats,"(2009-2010AlmeríaStats,)","([(/en/players/12421a7c/Diego-Alves, /en/count...",club
99,2009-2010##Osasuna-Stats,"(2009-2010OsasunaStats,)","([(/en/players/2fe713ed/Ricardo, /en/country/E...",club
100,2009-2010##Espanyol-Stats,"(2009-2010EspanyolStats,)","([(/en/players/70df8d7c/Luis-Garcia, /en/count...",club


In [25]:
numClubSeason = len(club['stats'])
top3_goal = []
for i in range(numClubSeason):
    club_season_std = club['stats'][i]['std']
    club_season_info = club['info'][i]['ClubName']
    club_name = club_season_info[9:-5]
    club_season = club_season_info[:9]
    goal = []
    for j in range(len(club_season_std)):
        player = club_season_std[j]['Player']
        nation = club_season_std[j]['Nation']
        pos = club_season_std[j]['Pos']
        age = club_season_std[j]['Age']
        gls = club_season_std[j]['Gls']
        goal.append((player,nation,pos,age,gls))
    goal.sort(key = lambda x:x[4],reverse = True)
    for idx in range(3):
        top3_goal.append((club_name,club_season,goal[idx][0],goal[idx][1],goal[idx][2],goal[idx][3],goal[idx][4]))

columns = ["ClubName","ClubSeason","Player","Nation","Position","Age","Goal"]
df_test = spark.createDataFrame(top3_goal,columns)
df_test.printSchema()
#### Save data to postgresql : TOP 3 CAU THU GHI BAN CUA MOI CLB TRONG MUA GIAI
df_test.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Top3Goal") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()
df_test.show()
 
    

root
 |-- ClubName: string (nullable = true)
 |-- ClubSeason: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Goal: string (nullable = true)



AnalysisException: Table or view 'Top3Goal' already exists. SaveMode: ErrorIfExists.

In [44]:

### TOP 5 CAU THU CO THOI GIAN THI DAU NHIEU NHAT CUA MOI CLB ####
numClubSeason = len(club['stats'])
top10_playtime = []
for i in range(numClubSeason):
    club_season_playtime = club['stats'][i]['playing_time']
    club_season_info = club['info'][i]['ClubName']
    if (club_season_playtime[0]['Player'] is None):
        break
    playtime = []
    for j in range(len(club_season_playtime)):
        player = club_season_playtime[j]['Player']
        nation = club_season_playtime[j]['Nation']
        pos = club_season_playtime[j]['Pos']
        age = club_season_playtime[j]['Age']
        minute = club_season_playtime[j]['Min']
        min_per_match = club_season_playtime[j]['Mn/MP']
        if(minute is not None):
            # print(i,player)
            minute = minute.replace(',','')
            minute = int(minute)
            playtime.append((player,nation,pos,age,minute,min_per_match))
    playtime.sort(key = lambda x:x[4],reverse = True)
    club_name = club_season_info[9:-5]
    club_season = club_season_info[:9]
    for idx in range(5):
        top10_playtime.append((club_name,club_season,playtime[idx][0],playtime[idx][1],playtime[idx][2],playtime[idx][3],playtime[idx][4],playtime[idx][5]))
    
columns = ["ClubName","ClubSeason","Player","Nation","Position","Age","Minutes","Mn/MP"]
df_test2 = spark.createDataFrame(top10_playtime,columns)
df_test2.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Top5Playtime") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()
df_test2.printSchema()
df_test2.show()
 
    

root
 |-- ClubName: string (nullable = true)
 |-- ClubSeason: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Minutes: long (nullable = true)
 |-- Mn/MP: string (nullable = true)

+---------+----------+--------------------+--------------------+--------+---+-------+-----+
| ClubName|ClubSeason|              Player|              Nation|Position|Age|Minutes|Mn/MP|
+---------+----------+--------------------+--------------------+--------+---+-------+-----+
|Salamanca| 1998-1999|/en/players/d859d...|/en/country/ESP/S...|      MF| 26|   3368|   89|
|Salamanca| 1998-1999|/en/players/911e7...|/en/country/ESP/S...|      MF| 24|   3232|   90|
|Salamanca| 1998-1999|/en/players/e6a5d...|/en/country/BRA/B...|      MF| 27|   2923|   89|
|Salamanca| 1998-1999|/en/players/0af98...|/en/country/ROU/R...|      GK| 30|   2917|   88|
|Salamanca| 1998-1999|/en/players/9cc18...|

In [45]:
player_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('uri', "mongodb://mongodb:27017/football_data.player") \
    .load()
    
player_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- ShortName: string (nullable = true)
 |    |-- FullName: void (nullable = true)
 |    |-- Position: string (nullable = true)
 |    |-- Height: void (nullable = true)
 |    |-- Weight: string (nullable = true)
 |    |-- Footed: string (nullable = true)
 |    |-- DOB: string (nullable = true)
 |    |-- Nationality: string (nullable = true)
 |    |-- Club: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- adv_goalkeeping: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Season: string (nullable = true)
 |    |    |    |-- Age: string (nullable = true)
 |    |    |    |-- Squad: string (nullable = true)
 |    |    |    |-- Country: string (nullable = true)
 |    |    |    |-- Comp: string (nullable = true)
 |    |    |    |-- LgRank: string (nullable = true)
 |    |    |    |-- 90s: string (nullable = true)
 |    |    |    |-- GA: string

In [46]:
player = player_df.toPandas()
player

,_id,info,stats,type
0,57e3f0c7,"(Nicolas Pépé, None, FW-MF (AM, right) ▪ , N...","(None, [(2014-2015, 19, /en/squads/69236f98/20...",player
1,00b77ecf,"(Nathan Fraser, None, None, None, None, None, ...","(None, None, None, None, [(0, 0.4, 16, /en/com...",player
2,aa8e289e,"(Nikola Vlašić, None, FW-MF (AM) ▪ , None, 1...","(None, [(2014-2015, 16, /en/squads/6777e16d/20...",player
3,f08490a6,"(Arnaut Danjuma, None, None, None, None, None,...","(None, [(2015-2016, 18, /en/squads/8e1ea572/20...",player
4,f9d409a1,"(Jimmy Morgan, None, None, None, None, None, N...","(None, None, None, None, [(0, 1.0, 15, /en/com...",player
...,...,...,...,...
224,6b625ac2,"(Illan Meslier, None, None, None, None, None, ...","([(2018-2019, 18, /en/squads/d2c87802/2018-201...",player
225,819aa8e7,"(Ayoze Pérez, None, None, None, None, None, No...","(None, [(2013-2014, 20, /en/squads/27cc9c62/20...",player
226,ca45605e,"(Patson Daka, None, FW ▪ , None, 180cm, Rig...","(None, [(2017-2018, 18, /en/squads/50f2a074/20...",player
227,ee38d9c5,"(James Maddison, None, None, None, None, None,...","(None, [(2014-2015, 17, /en/squads/f7e3dfe9/20...",player


In [53]:
numPlayer = len(player['_id'])
goal_season = []
ast_season = []
overall = []
match_season = []

for i in range(numPlayer):
    player_info = player['info'][i]['ShortName']
    sum_goal = 0
    sum_match = 0
    sum_ast = 0
    for j in range(len(player['stats'][i]['std'])):
        season = player['stats'][i]['std'][j]['Season']
        goal = player['stats'][i]['std'][j]['Gls']
        ast = player['stats'][i]['std'][j]['Ast']
        match = player['stats'][i]['std'][j]['MP']
        start = player['stats'][i]['std'][j]['Starts']
        if goal is not None and ast is not None:
            goal = int(float(goal))
            ast = int(float(ast))
            sum_goal += goal
            sum_ast += ast
            sum_match += int(float(match))
            goal_season.append([player_info,season,goal])
            ast_season.append([player_info,season,ast])
            match_season.append([player_info,season,match])
        
    overall.append([player_info,sum_goal,sum_ast,sum_match])

### THONG SO GHI BAN QUA CAC MUA CUA CAU THU
columns = ["Player","Season","Goal"]
goal_season_df = spark.createDataFrame(goal_season,columns)
goal_season_df.show()
goal_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "PlayerGoal") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

### THONG SO KIEN TAO CUA CAU THU QUA CAC MUA
columns = ["Player","Season","Assist"]
ast_season_df = spark.createDataFrame(ast_season,columns)
ast_season_df.show()
ast_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "PlayerAssist") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

### THONG SO VE SO TRAN THI DAU CUA CAU THU QUA CAC MUA
columns = ["Player","Season","Matches"]
matches_season_df = spark.createDataFrame(match_season,columns)
matches_season_df.show()
matches_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "PlayerMatches") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

### THONG SO VE TOAN BO SO BAN THANG KIEN TAO SỐ TRẬN CỦA CẦU THỦ TRONG SỰ NGHIỆP
columns = ["Player","Goals","Assist","Matches"]
overall_df = spark.createDataFrame(overall,columns)      
overall_df.show()
overall_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "PlayerOverall") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

        
    
    

+-------------+---------+----+
|       Player|   Season|Goal|
+-------------+---------+----+
| Nicolas Pépé|2014-2015|   0|
| Nicolas Pépé|2016-2017|   3|
| Nicolas Pépé|2017-2018|  13|
| Nicolas Pépé|2018-2019|  22|
| Nicolas Pépé|2019-2020|   5|
| Nicolas Pépé|2020-2021|  10|
| Nicolas Pépé|2021-2022|   1|
| Nicolas Pépé|2022-2023|   6|
|Nathan Fraser|2021-2022|   1|
|Nathan Fraser|2022-2023|   2|
|Nikola Vlašić|2014-2015|   3|
|Nikola Vlašić|2015-2016|   1|
|Nikola Vlašić|2016-2017|   4|
|Nikola Vlašić|2017-2018|   3|
|Nikola Vlašić|2017-2018|   0|
|Nikola Vlašić|2017-2018|   0|
|Nikola Vlašić|2018-2019|   5|
|Nikola Vlašić|2019-2020|  12|
|Nikola Vlašić|2020-2021|  11|
|Nikola Vlašić|2021-2022|   0|
+-------------+---------+----+
only showing top 20 rows

+-------------+---------+------+
|       Player|   Season|Assist|
+-------------+---------+------+
| Nicolas Pépé|2014-2015|     0|
| Nicolas Pépé|2016-2017|     1|
| Nicolas Pépé|2017-2018|     4|
| Nicolas Pépé|2018-2019|    11|

In [58]:
numPlayer = len(player['_id'])
shooting_season = []

for i in range(numPlayer):
    player_info = player['info'][i]['ShortName']
    sum_goal = 0
    sum_match = 0
    sum_ast = 0
    if player['stats'][i]['shooting'] is not None:
        for j in range(len(player['stats'][i]['shooting'])):
            season = player['stats'][i]['shooting'][j]['Season']
            gls = player['stats'][i]['shooting'][j]['Gls']
            g_sot = player['stats'][i]['shooting'][j]['G/SoT']
            xG = player['stats'][i]['shooting'][j]['xG']

            if xG is not None:
                # g_sot = float(g_sot)
                xG = float(xG)
                shooting_season.append([player_info,season,gls,g_sot,xG])
            # xG_season.append([player_info,season,xG])
            
### Thong so ghi ban cua cau thu
columns = ["Player","Season","Goal","G/SoT","Expected Goal"]
shooting_df = spark.createDataFrame(shooting_season,columns)
shooting_df.show()
shooting_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Player_Shooting") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

        
    
    

+--------------+---------+----+-----+-------------+
|        Player|   Season|Goal|G/SoT|Expected Goal|
+--------------+---------+----+-----+-------------+
|  Nicolas Pépé|2017-2018|  13| 0.46|          8.2|
|  Nicolas Pépé|2018-2019|  22| 0.25|         21.1|
|  Nicolas Pépé|2019-2020|   5| 0.24|          4.9|
|  Nicolas Pépé|2020-2021|  10| 0.47|          5.8|
|  Nicolas Pépé|2021-2022|   1| 0.14|          2.7|
|  Nicolas Pépé|2022-2023|   6| 0.20|          7.3|
| Nikola Vlašić|2017-2018|   0| 0.00|          0.3|
| Nikola Vlašić|2021-2022|   1| 0.33|          1.0|
| Nikola Vlašić|2022-2023|   4| 0.27|          3.7|
|Arnaut Danjuma|2019-2020|   0| 0.00|          1.2|
|Arnaut Danjuma|2020-2021|  15| 0.47|          7.5|
|Arnaut Danjuma|2021-2022|  10| 0.28|         11.8|
|Arnaut Danjuma|2022-2023|   2| 0.25|          2.7|
|   Lyle Taylor|2019-2020|  11| 0.39|          9.7|
|   Lyle Taylor|2020-2021|   4| 0.21|          7.4|
|   Lyle Taylor|2021-2022|   5| 0.30|          4.3|
|   Lyle Tay

In [62]:
numPlayer = len(player['_id'])
gsc_season = []
### GOAL AND SHOT CREATION
for i in range(numPlayer):
    player_info = player['info'][i]['ShortName']
    if player['stats'][i]['goal_shot_creation'] is not None:
        for j in range(len(player['stats'][i]['goal_shot_creation'])):
            season = player['stats'][i]['goal_shot_creation'][j]['Season']
            age = player['stats'][i]['goal_shot_creation'][j]['Age']
            squad = player['stats'][i]['goal_shot_creation'][j]['Squad']
            sca = player['stats'][i]['goal_shot_creation'][j]['SCA']
            sca90 = player['stats'][i]['goal_shot_creation'][j]['SCA90']
            gca = player['stats'][i]['goal_shot_creation'][j]['GCA']
            gca90 = player['stats'][i]['goal_shot_creation'][j]['GCA90']

            if sca is not None:
                sca = int(sca)
                gca = int(gca)
                sca90 = float(sca90)
                gca90 = float(gca90)
                gsc_season.append((player_info,season,age,squad,sca,sca90,gca,gca90))
            
            
        
    

columns = ["Player","Season","Age","Squad","SCA","SCA90","GCA","GCA90"]
gsc_df = spark.createDataFrame(gsc_season,columns)
gsc_df.printSchema()
gsc_df.show()
gsc_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Player_GoalCreation") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()
        
    
    

root
 |-- Player: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- SCA: long (nullable = true)
 |-- SCA90: double (nullable = true)
 |-- GCA: long (nullable = true)
 |-- GCA90: double (nullable = true)

+--------------+---------+---+--------------------+---+-----+---+-----+
|        Player|   Season|Age|               Squad|SCA|SCA90|GCA|GCA90|
+--------------+---------+---+--------------------+---+-----+---+-----+
|  Nicolas Pépé|2017-2018| 22|/en/squads/cb188c...| 99| 3.14|  9| 0.29|
|  Nicolas Pépé|2018-2019| 23|/en/squads/cb188c...|177| 4.79| 36| 0.97|
|  Nicolas Pépé|2019-2020| 24|/en/squads/18bb7c...| 71| 3.18|  9|  0.4|
|  Nicolas Pépé|2020-2021| 25|/en/squads/18bb7c...| 43| 2.39|  8| 0.45|
|  Nicolas Pépé|2021-2022| 26|/en/squads/18bb7c...| 36| 4.66|  4| 0.52|
|  Nicolas Pépé|2022-2023| 27|/en/squads/132ebc...| 41| 2.94|  6| 0.43|
| Nikola Vlašić|2017-2018| 19|/en/squads/d3fd31...| 23| 3.

In [4]:
league_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('uri', "mongodb://mongodb:27017/football_data.league") \
    .load()
    
league_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- LeagueName: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- goal_shot_creation: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Squad: string (nullable = true)
 |    |    |    |-- # Pl: string (nullable = true)
 |    |    |    |-- 90s: string (nullable = true)
 |    |    |    |-- SCA: string (nullable = true)
 |    |    |    |-- SCA90: string (nullable = true)
 |    |    |    |-- PassLive: string (nullable = true)
 |    |    |    |-- PassDead: string (nullable = true)
 |    |    |    |-- Drib: string (nullable = true)
 |    |    |    |-- Sh: string (nullable = true)
 |    |    |    |-- Fld: string (nullable = true)
 |    |    |    |-- Def: string (nullable = true)
 |    |    |    |-- GCA: string (nullable = true)
 |    |    |    |-- GCA90: string (nullable = true)
 |    |    |    |-- PassLive2: string (nullable = true)
 |    |   

In [5]:
league = league_df.toPandas()
league

,_id,info,stats,type
0,9##Premier-League-Stats,"(2022-2023PremierLeagueStats,)","([(/en/squads/18bb7c10/Arsenal-Stats, 24, 19.0...",league
1,9##2009-2010,"(2009-2010PremierLeagueStats,)","(None, [(/en/squads/18bb7c10/2009-2010/Arsenal...",league
2,9##2010-2011,"(2010-2011PremierLeagueStats,)","(None, [(/en/squads/18bb7c10/2010-2011/Arsenal...",league
3,9##2011-2012,"(2011-2012PremierLeagueStats,)","(None, [(/en/squads/18bb7c10/2011-2012/Arsenal...",league
4,9##2012-2013,"(2012-2013PremierLeagueStats,)","(None, [(/en/squads/18bb7c10/2012-2013/Arsenal...",league
...,...,...,...,...
101,11##2012-2013,"(2012-2013SerieAStats,)","(None, [(/en/squads/922493f3/2012-2013/Atalant...",league
102,11##2013-2014,"(2013-2014SerieAStats,)","(None, [(/en/squads/922493f3/2013-2014/Atalant...",league
103,11##2014-2015,"(2014-2015SerieAStats,)","(None, [(/en/squads/922493f3/2014-2015/Atalant...",league
104,11##2015-2016,"(2015-2016SerieAStats,)","(None, [(/en/squads/922493f3/2015-2016/Atalant...",league


In [66]:
numLeague = len(league['_id'])
gls_season = []


for i in range(numLeague):
    league_info = league['info'][i]['LeagueName']
    leaguename = league_info[9:-5]
    leagueseason = league_info[:9]
    gls = []
    if league['stats'][i]['std'] is not None:
        for j in range(len(league['stats'][i]['std'])):
            squad = league['stats'][i]['std'][j]['Squad']
            goal = league['stats'][i]['std'][j]['Gls']
            xgoal = league['stats'][i]['std'][j]['xG']
            if squad is not None:
                gls_season.append((leaguename,leagueseason,squad,goal,xgoal))
    # gls_season.append([leaguename,leagueseason,gls])

columns = ["League","Season","Squad","Goals","Expected Goals"]
gls_season_df = spark.createDataFrame(gls_season,columns)
gls_season_df.show()
gls_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "League_Goal") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

    

+-------------+---------+--------------------+-----+--------------+
|       League|   Season|               Squad|Goals|Expected Goals|
+-------------+---------+--------------------+-----+--------------+
|PremierLeague|2022-2023|/en/squads/18bb7c...|   43|          37.3|
|PremierLeague|2022-2023|/en/squads/860229...|   23|          24.6|
|PremierLeague|2022-2023|/en/squads/4ba7cb...|   19|          15.2|
|PremierLeague|2022-2023|/en/squads/cd0518...|   31|          27.4|
|PremierLeague|2022-2023|/en/squads/d07537...|   33|          29.1|
|PremierLeague|2022-2023|/en/squads/cff3d9...|   22|          23.8|
|PremierLeague|2022-2023|/en/squads/47c64c...|   17|          18.3|
|PremierLeague|2022-2023|/en/squads/d3fd31...|   14|          20.4|
|PremierLeague|2022-2023|/en/squads/fd9621...|   29|          28.5|
|PremierLeague|2022-2023|/en/squads/5bfb96...|   25|          25.8|
|PremierLeague|2022-2023|/en/squads/a2d435...|   26|          22.7|
|PremierLeague|2022-2023|/en/squads/822bd0...|  

In [78]:
numLeague = len(league['_id'])
best5_ga_season = []
worst5_ga_season = []
top5_save_season = []
top5_cleansheet_season = []
for i in range(numLeague):
    league_info = league['info'][i]['LeagueName']
    leaguename = league_info[9:-5]
    leagueseason = league_info[:9]
    ga = []
    cs = []
    save = []
    if league['stats'][i]['goalkeeping'] is not None:
        for j in range(len(league['stats'][i]['goalkeeping'])):
            squad = league['stats'][i]['goalkeeping'][j]['Squad']
            goal_againist = int(league['stats'][i]['goalkeeping'][j]['GA'])
            cs_percentage = float(league['stats'][i]['goalkeeping'][j]['CS%'])
            save_percentage = float(league['stats'][i]['goalkeeping'][j]['Save%'])
            if squad is not None and save_percentage is not None:
                ga.append((squad,goal_againist))
                cs.append((squad,cs_percentage))
                save.append((squad,save_percentage))
            
        ga.sort(key = lambda x:x[1],reverse= False)
        worst5_ga = ga[-5:]
        best5_ga = ga[:5]
        cs.sort(key = lambda x:x[1],reverse= True)
        cs5 = cs[:5]
        save.sort(key = lambda x:x[1],reverse= True)
        save5 = save[:5]
    for idx in range(5):
        if best5_ga is not None:
            best5_ga_season.append([leaguename,leagueseason,best5_ga[idx][0],best5_ga[idx][1]])
            worst5_ga_season.append([leaguename,leagueseason,worst5_ga[idx][0],worst5_ga[idx][1]])
        if save is not None:
            top5_save_season.append([leaguename,leagueseason,save5[idx][0],save5[idx][1]])
        if cs is not None:
            top5_cleansheet_season.append([leaguename,leagueseason,cs5[idx][0],cs5[idx][1]])

# print(best5_ga_season)
# 5 doi thủng lưới ít nhất
columns = ["League Name","Season","Squad","Goal Againist"]
best5_ga_season_df = spark.createDataFrame(best5_ga_season,columns)
best5_ga_season_df.show()
best5_ga_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Best5_GoalAgainist") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

#5 đội thủng lưới nhiều nhất
columns = ["League Name","Season","Squad","Goal Againist"]
worst5_ga_season_df = spark.createDataFrame(worst5_ga_season,columns)
worst5_ga_season_df.show()
worst5_ga_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Worst5_GoalAgainist") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

#5 đội có tỉ lệ cứu thua cao nhất
columns = ["League Name","Season","Squad","Save Percentage"]
top5_save_season_df = spark.createDataFrame(top5_save_season,columns)
top5_save_season_df.show()
top5_save_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Top5_SavePercent") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

#5 đội giữ sạch lưới tốt nhất
columns = ["League Name","Season","Squad","Clean sheet percentage"]
top5_cleansheet_season_df = spark.createDataFrame(top5_cleansheet_season,columns)
top5_cleansheet_season_df.show()
top5_cleansheet_season_df.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "Top5_CLeasheet") \
    .option("user", "test1") \
    .option("password", "test1") \
    .save()

+-------------+---------+--------------------+-------------+
|  League Name|   Season|               Squad|Goal Againist|
+-------------+---------+--------------------+-------------+
|PremierLeague|2022-2023|/en/squads/b2b47a...|           11|
|PremierLeague|2022-2023|/en/squads/18bb7c...|           16|
|PremierLeague|2022-2023|/en/squads/b8fd03...|           20|
|PremierLeague|2022-2023|/en/squads/cff3d9...|           21|
|PremierLeague|2022-2023|/en/squads/822bd0...|           25|
|PremierLeague|2009-2010|/en/squads/195388...|           28|
|PremierLeague|2009-2010|/en/squads/cff3d9...|           32|
|PremierLeague|2009-2010|/en/squads/822bd0...|           35|
|PremierLeague|2009-2010|/en/squads/860229...|           39|
|PremierLeague|2009-2010|/en/squads/18bb7c...|           41|
|PremierLeague|2010-2011|/en/squads/cff3d9...|           33|
|PremierLeague|2010-2011|/en/squads/b8fd03...|           33|
|PremierLeague|2010-2011|/en/squads/195388...|           37|
|PremierLeague|2010-2011

In [31]:
numLeague = len(league['_id'])
best5_ga_season = []
worst5_ga_season = []
top5_save_season = []
top5_cleansheet_season = []
for i in range(numLeague):
    league_info = league['info'][i]['LeagueName']
    leaguename = league_info[9:-5]
    leagueseason = league_info[:9]
    ga = []
    cs = []
    save = []
    if league['stats'][i]['goalkeeping'] is not None:
        for j in range(len(league['stats'][i]['goalkeeping'])):
            squad = league['stats'][i]['goalkeeping'][j]['Squad']
            goal_againist = int(league['stats'][i]['goalkeeping'][j]['GA'])
            cs_percentage = float(league['stats'][i]['goalkeeping'][j]['CS%'])
            save_percentage = float(league['stats'][i]['goalkeeping'][j]['Save%'])
            if squad is not None:
                ga.append((squad,goal_againist))
                cs.append((squad,cs_percentage))
                save.append((squad,save_percentage))
            
        ga.sort(key = lambda x:x[1],reverse= False)
        worst5_ga = ga[-5:]
        best5_ga = ga[:5]
        cs.sort(key = lambda x:x[1],reverse= True)
        cs= cs[:5]
        save.sort(key = lambda x:x[1],reverse= True)
        save = save[:5]
    best5_ga_season.append([leaguename,leagueseason,best5_ga])
    worst5_ga_season.append([leaguename,leagueseason,worst5_ga])
    top5_save_season.append([leaguename,leagueseason,save])
    top5_cleansheet_season.append([leaguename,leagueseason,cs])

# print(best5_ga_season)
columns = ["League Name","Season","Best 5 GA"]
best5_ga_season_df = spark.createDataFrame(best5_ga_season,columns)
best5_ga_season_df.show()

columns = ["League Name","Season","Worst 5 GA"]
worst5_ga_season_df = spark.createDataFrame(worst5_ga_season,columns)
worst5_ga_season_df.show()

columns = ["League Name","Season","Top 5 Save"]
top5_save_season_df = spark.createDataFrame(top5_save_season,columns)
top5_save_season_df.show()

columns = ["League Name","Season","Top 5 Clean sheet"]
top5_cleansheet_season_df = spark.createDataFrame(top5_cleansheet_season,columns)
top5_cleansheet_season_df.show()

+--------------+---------+--------------------+
|   League Name|   Season|           Best 5 GA|
+--------------+---------+--------------------+
| PremierLeague|2022-2023|[{/en/squads/b2b4...|
| PremierLeague|2009-2010|[{/en/squads/1953...|
| PremierLeague|2010-2011|[{/en/squads/cff3...|
| PremierLeague|2011-2012|[{/en/squads/b8fd...|
| PremierLeague|2012-2013|[{/en/squads/b8fd...|
| PremierLeague|2013-2014|[{/en/squads/cff3...|
| PremierLeague|2014-2015|[{/en/squads/cff3...|
| PremierLeague|2015-2016|[{/en/squads/1953...|
| PremierLeague|2016-2017|[{/en/squads/361c...|
| PremierLeague|2017-2018|[{/en/squads/b8fd...|
| PremierLeague|2018-2019|[{/en/squads/822b...|
| PremierLeague|2019-2020|[{/en/squads/822b...|
|FootballLeague|1888-1889|[{/en/squads/822b...|
|        LaLiga|1988-1989|[{/en/squads/822b...|
|        SerieA|1988-1989|[{/en/squads/822b...|
|     Division1|1995-1996|[{/en/squads/822b...|
|    Bundesliga|1988-1989|[{/en/squads/822b...|
|    Bundesliga|1989-1990|[{/en/squads/8